In [10]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

# Scoring rules
https://www.espn.com/fantasy/football/ffl/story?page=fflrulesstandardscoring

Kicker:
- 5pt - 50+ yd FG
- 4pt - 40-49 yd FG
- 3pt - 0-39 yd FG
- 2pt - 2pt conversion
- 1pt - XP
- -2pt - missed FG (0-39 yd)
- -1pt - missed FG (40-49 yd)



In [4]:
df_pbp = pd.read_csv('data/pbp_1999_2024.csv')

/var/folders/mc/0hr3l37s7_z1722wyt381c6m0000gn/T/ipykernel_6064/2498530999.py:1: DtypeWarning: Columns (37,38,46,180,181,183,184,190,191,194,195,198,199,204,205,206,207,208,209,210,211,212,213,214,215,219,220,221,223,225,227,234,235,236,237,238,239,244,245,246,249,250,254,255,256,261,263,264,267,268,269,270,284,285,293,294,295,296,297,300,302,303,304,307,333,374,376,377,378,380,382,383,384,390,391,392) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pbp = pd.read_csv('data/pbp_1999_2024.csv')


,Unnamed: 0,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,...,offense_players,defense_players,n_offense,n_defense,ngs_air_yards,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type
0,0,1.0,2024_01_ARI_BUF,2.024091e+09,BUF,ARI,REG,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40.0,2024_01_ARI_BUF,2.024091e+09,BUF,ARI,REG,1,ARI,away,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,61.0,2024_01_ARI_BUF,2.024091e+09,BUF,ARI,REG,1,ARI,away,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,83.0,2024_01_ARI_BUF,2.024091e+09,BUF,ARI,REG,1,ARI,away,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,108.0,2024_01_ARI_BUF,2.024091e+09,BUF,ARI,REG,1,ARI,away,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_pbp.head()

,Unnamed: 0,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,

In [62]:
df_kicker_pbp = df_pbp[df_pbp['kicker_player_name'].notnull()]
# df_kicker_pbp = df_pbp[(df_pbp['play_type'] == 'field_goal') | (df_pbp['play_type'] == 'extra_point')]

In [63]:
df_kicker_pbp['game_date'] = pd.to_datetime(df_kicker_pbp['game_date'])

/var/folders/mc/0hr3l37s7_z1722wyt381c6m0000gn/T/ipykernel_6064/3617461328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kicker_pbp['game_date'] = pd.to_datetime(df_kicker_pbp['game_date'])


In [64]:
df_kicker_pbp.head()

,Unnamed: 0,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,

In [65]:
# XP
df_kicker_pbp['xp_distance'] = np.where(df_kicker_pbp['game_date'].dt.year < 2015, 19, 33)
df_kicker_pbp["xp_attempt"] = df_kicker_pbp["extra_point_result"].notnull()
df_kicker_pbp["xp_made"] = df_kicker_pbp["extra_point_result"] == "good"

df_kicker_pbp["xp_made_33y"] = df_kicker_pbp["xp_made"] & (df_kicker_pbp["xp_distance"] == 33)
df_kicker_pbp["xp_made_19y"] = df_kicker_pbp["xp_made"] & (df_kicker_pbp["xp_distance"] == 19)
df_kicker_pbp["xp_attempt_33y"] = df_kicker_pbp["xp_attempt"] & (df_kicker_pbp["xp_distance"] == 33)
df_kicker_pbp["xp_attempt_19y"] = df_kicker_pbp["xp_attempt"] & (df_kicker_pbp["xp_distance"] == 19)


# FG
df_kicker_pbp["50+_fg_made"] = (df_kicker_pbp["field_goal_result"] == "made") & (df_kicker_pbp["kick_distance"] >= 50)
df_kicker_pbp["40-49_fg_made"] = (df_kicker_pbp["field_goal_result"] == "made") & (df_kicker_pbp["kick_distance"] >= 40) & (df_kicker_pbp["kick_distance"] < 50)
df_kicker_pbp["0-39_fg_made"] = (df_kicker_pbp["field_goal_result"] == "made") & (df_kicker_pbp["kick_distance"] < 40)
df_kicker_pbp["missed_fg_0-39"] = (df_kicker_pbp["field_goal_result"] == "missed") & (df_kicker_pbp["kick_distance"] < 40)
df_kicker_pbp["missed_fg_40-49"] = (df_kicker_pbp["field_goal_result"] == "missed") & (df_kicker_pbp["kick_distance"] >= 40) & (df_kicker_pbp["kick_distance"] < 50)
df_kicker_pbp["missed_fg_50+"] = (df_kicker_pbp["field_goal_result"] == "missed") & (df_kicker_pbp["kick_distance"] >= 50)

# df_kicker_pbp["2pt_conv"] = df_kicker_pbp["play_type"] == "two_point_conversion"

df_kicker_pbp["fantasy_points"] = df_kicker_pbp["50+_fg_made"] * 5 + df_kicker_pbp["40-49_fg_made"] * 4 + df_kicker_pbp["0-39_fg_made"] * 3 + df_kicker_pbp["xp_made"] * 1 + df_kicker_pbp["missed_fg_0-39"] * -2 + df_kicker_pbp["missed_fg_40-49"] * -1




/var/folders/mc/0hr3l37s7_z1722wyt381c6m0000gn/T/ipykernel_6064/2853644080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kicker_pbp['xp_distance'] = np.where(df_kicker_pbp['game_date'].dt.year < 2015, 19, 33)
/var/folders/mc/0hr3l37s7_z1722wyt381c6m0000gn/T/ipykernel_6064/2853644080.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kicker_pbp["xp_attempt"] = df_kicker_pbp["extra_point_result"].notnull()
/var/folders/mc/0hr3l37s7_z1722wyt381c6m0000gn/T/ipykernel_6064/2853644080.py:4: SettingWit

## List of kicker stats to aggregate

Given:
- game_id
- home_team
- away_team
- week
- posteam
- defteam
<!-- - yardline_100 -->
- game_date
<!-- - play_type -->
- field_goal_result
- kick_distance
- extra_point_result
- field_goal_attempt
- kicker_player_name
- kicker_player_id
- stadium
- weather
- roof
- surface
- temp
- wind

Calculated:
- fantasy_points
- 50+_fg_made
- 40-49_fg_made
- 0-39_fg_made
- missed_fg_0-39
- missed_fg_40-49
- missed_fg_50+
- xp_attempt
- xp_made
- xp_distance
<!-- - xp_made_15yl
- xp_made_2yl -->

In [73]:
df_kicker_game_level = df_kicker_pbp.groupby(['game_id', 'kicker_player_name', 'kicker_player_id']).agg({
    # Game level
    'game_id': 'first',
    'home_team': 'first',
    'away_team': 'first',
    'week': 'first',
    'posteam': 'first',
    'defteam': 'first',
    'game_date': 'first',
    'stadium': 'first',
    'weather': 'first',
    'roof': 'first',
    'surface': 'first',
    'temp': 'first',
    'wind': 'first',
    # Play level
    'fantasy_points': 'sum',
    '50+_fg_made': 'sum',
    '40-49_fg_made': 'sum',
    '0-39_fg_made': 'sum',
    'missed_fg_0-39': 'sum',
    'missed_fg_40-49': 'sum',
    'missed_fg_50+': 'sum',
    # 'xp_attempt': 'sum',
    # 'xp_made': 'sum',
    # 'xp_distance': 'first',
    'xp_attempt_19y': 'sum',
    'xp_made_19y': 'sum',
    'xp_attempt_33y': 'sum',
    'xp_made_33y': 'sum',
})

In [74]:
df_kicker_game_level["home"] = df_kicker_game_level["home_team"] == df_kicker_game_level["posteam"]
df_kicker_game_level.drop(columns=['home_team', 'away_team'], inplace=True)

In [75]:
df_kicker_game_level.head()

game_id  week  \
game_id         kicker_player_name kicker_player_id                          
1999_01_ARI_PHI C.Jacke            00-0008080        1999_01_ARI_PHI     1   
                D.Akers            00-0000108        1999_01_ARI_PHI     1   
                N.Johnson          00-0008593        1999_01_ARI_PHI     1   
1999_01_BUF_IND M.Vanderjagt       00-0016830        1999_01_BUF_IND     1   
                S.Christie         00-0002975        1999_01_BUF_IND     1   

                                                    posteam defteam  \
game_id         kicker_player_name kicker_player_id                   
1999_01_ARI_PHI C.Jacke            00-0008080           PHI     ARI   
                D.Akers            00-0000108           ARI     PHI   
                N.Johnson          00-0008593           PHI     ARI   
1999_01_BUF_IND M.Vanderjagt       00-0016830           BUF     IND   
                S.Christie         00-0002975           BUF     IND   

                                                     game_date stadium  \
game_id         kicker_player_name kicker_player_id                      
1999_01_ARI_PHI C.Jacke            00-0008080       1999-09-12    None   
                D.Akers            00-0000108       1999-09-12    None   
                N.Johnson          00-0008593       1999-09-12    None   
1999_01_BUF_IND M.Vanderjagt       00-0016830       1999-09-12    None   
                S.Christie         00-0002975       1999-09-12    None   

                                                    weather      roof  \
game_id         kicker_player_name kicker_player_id                     
1999_01_ARI_PHI C.Jacke            00-0008080          None  outdoors   
                D.Akers            00-0000108          None  outdoors   
                N.Johnson          00-0008593          None  outdoors   
1999_01_BUF_IND M.Vanderjagt       00-0016830          None      dome   
                S.Christie         00-0002975          None      dome   

                                                       surface  temp  wind  \
game_id         kicker_player_name kicker_player_id                          
1999_01_ARI_PHI C.Jacke            00-0008080        astroturf  75.0   3.0   
                D.Akers            00-0000108        astroturf  75.0   3.0   
                N.Johnson          00-0008593        astroturf  75.0   3.0   
1999_01_BUF_IND M.Vanderjagt       00-0016830        astroturf   NaN   NaN   
                S.Christie         00-0002975        astroturf   NaN   NaN   

                                                     fantasy_points  \
game_id         kicker_player_name kicker_player_id                   
1999_01_ARI_PHI C.Jacke            00-0008080                    13   
                D.Akers            00-0000108                     0   
                N.Johnson          00-0008593                     6   
1999_01_BUF_IND M.Vanderjagt       00-0016830                     7   
                S.Christie         00-0002975                     6   

                                                     50+_fg_made  \
game_id         kicker_player_name kicker_player_id                
1999_01_ARI_PHI C.Jacke            00-0008080                  0   
                D.Akers            00-0000108                  0   
                N.Johnson          00-0008593                  0   
1999_01_BUF_IND M.Vanderjagt       00-0016830                  0   
                S.Christie         00-0002975                  0   

                                                     40-49_fg_made  \
game_id         kicker_player_name kicker_player_id                  
1999_01_ARI_PHI C.Jacke            00-0008080                    0   
                D.Akers            00-0000108                    0   
                N.Johnson          00-0008593                    0   
1999_01_BUF_IND M.Vanderjagt       00-0016830                    0   
                S.Christie         00-00029

## Predictors to calculate
Aggregate each point-earning stat at the following intervals:
- Past 5 games
- Season
- Career (kicker only)

Calculate the above for kickers, offenses, and defenses as well.

Calculate # games played by player in career (ie sample size).
Calculate # games in season (not quite same as week # -- injuries, byes)